In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("First_App").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/17 08:55:41 WARN Utils: Your hostname, nishal resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/06/17 08:55:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/17 08:55:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
film_df = spark.read.csv("./data/film.csv", header=True, inferSchema=True)
film_actor_df = spark.read.csv("./data/film_actor.csv", header=True, inferSchema=True)
actor_df = spark.read.csv("./data/actor.csv", header=True, inferSchema=True)
film_category_df = spark.read.csv("./data/film_category.csv", header=True, inferSchema=True)
category_df = spark.read.csv("./data/category.csv", inferSchema=True, header=True)

In [8]:
film_df.printSchema()
film_actor_df.printSchema()
actor_df.printSchema()
film_category_df.printSchema()
category_df.printSchema()

root
 |-- film_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- language_id: integer (nullable = true)
 |-- original_language_id: string (nullable = true)
 |-- rental_duration: integer (nullable = true)
 |-- rental_rate: double (nullable = true)
 |-- length: integer (nullable = true)
 |-- replacement_cost: double (nullable = true)
 |-- rating: string (nullable = true)
 |-- special_features: string (nullable = true)
 |-- last_update: timestamp (nullable = true)

root
 |-- actor_id: integer (nullable = true)
 |-- film_id: integer (nullable = true)
 |-- last_update: timestamp (nullable = true)

root
 |-- actor_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- last_update: timestamp (nullable = true)

root
 |-- film_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- last_update: timesta

In [9]:
new_actor = actor_df.join(film_actor_df, actor_df.actor_id == film_actor_df.actor_id, "right").select(actor_df.actor_id, film_actor_df.film_id, actor_df.first_name, actor_df.last_name)

In [11]:
new_actor.show(10)

+--------+-------+----------+---------+
|actor_id|film_id|first_name|last_name|
+--------+-------+----------+---------+
|       1|      1|  PENELOPE|  GUINESS|
|       1|     23|  PENELOPE|  GUINESS|
|       1|     25|  PENELOPE|  GUINESS|
|       1|    106|  PENELOPE|  GUINESS|
|       1|    140|  PENELOPE|  GUINESS|
|       1|    166|  PENELOPE|  GUINESS|
|       1|    277|  PENELOPE|  GUINESS|
|       1|    361|  PENELOPE|  GUINESS|
|       1|    438|  PENELOPE|  GUINESS|
|       1|    499|  PENELOPE|  GUINESS|
+--------+-------+----------+---------+
only showing top 10 rows



In [12]:
new_actor = new_actor.withColumnsRenamed({"first_name": "actor_first_name", "last_name": "actor_last_name"})

new_actor.show(10)

+--------+-------+----------------+---------------+
|actor_id|film_id|actor_first_name|actor_last_name|
+--------+-------+----------------+---------------+
|       1|      1|        PENELOPE|        GUINESS|
|       1|     23|        PENELOPE|        GUINESS|
|       1|     25|        PENELOPE|        GUINESS|
|       1|    106|        PENELOPE|        GUINESS|
|       1|    140|        PENELOPE|        GUINESS|
|       1|    166|        PENELOPE|        GUINESS|
|       1|    277|        PENELOPE|        GUINESS|
|       1|    361|        PENELOPE|        GUINESS|
|       1|    438|        PENELOPE|        GUINESS|
|       1|    499|        PENELOPE|        GUINESS|
+--------+-------+----------------+---------------+
only showing top 10 rows



In [14]:
new_category = category_df.join(film_category_df, film_category_df.category_id == category_df.category_id, "right").select(category_df.category_id, film_category_df.film_id, category_df.name)

new_category.show(10)

+-----------+-------+-----------+
|category_id|film_id|       name|
+-----------+-------+-----------+
|          6|      1|Documentary|
|         11|      2|     Horror|
|          6|      3|Documentary|
|         11|      4|     Horror|
|          8|      5|     Family|
|          9|      6|    Foreign|
|          5|      7|     Comedy|
|         11|      8|     Horror|
|         11|      9|     Horror|
|         15|     10|     Sports|
+-----------+-------+-----------+
only showing top 10 rows



In [15]:
new_category = new_category.withColumnRenamed("name", "category")

In [16]:
new_category.show(10)

+-----------+-------+-----------+
|category_id|film_id|   category|
+-----------+-------+-----------+
|          6|      1|Documentary|
|         11|      2|     Horror|
|          6|      3|Documentary|
|         11|      4|     Horror|
|          8|      5|     Family|
|          9|      6|    Foreign|
|          5|      7|     Comedy|
|         11|      8|     Horror|
|         11|      9|     Horror|
|         15|     10|     Sports|
+-----------+-------+-----------+
only showing top 10 rows



In [20]:
new_films = film_df.join(new_actor, new_actor.film_id == film_df.film_id, "left").select(
film_df.film_id,
film_df.title,
film_df.description,
film_df.release_year,
film_df.language_id,
film_df.original_language_id,
film_df.rental_duration,
film_df.rental_rate,
film_df.length,
film_df.replacement_cost,
film_df.rating,
film_df.special_features,
new_actor.actor_first_name,
new_actor.actor_last_name,
)

new_films.printSchema()
new_films.show(10)

root
 |-- film_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- language_id: integer (nullable = true)
 |-- original_language_id: string (nullable = true)
 |-- rental_duration: integer (nullable = true)
 |-- rental_rate: double (nullable = true)
 |-- length: integer (nullable = true)
 |-- replacement_cost: double (nullable = true)
 |-- rating: string (nullable = true)
 |-- special_features: string (nullable = true)
 |-- actor_first_name: string (nullable = true)
 |-- actor_last_name: string (nullable = true)

+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+----------------+---------------+
|film_id|           title|         description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|    special_fea